## Setting

In [3]:
# import os
# os.makedirs("./data/news", exist_ok=True)
# os.makedirs("./data/paper", exist_ok=True)
# os.makedirs("./corpus", exist_ok=True)
# os.makedirs("./tokenizer/", exist_ok=True)
# os.makedirs("./checkpoints/", exist_ok=True)
# os.makedirs("./outputs/", exist_ok=True)
# os.makedirs("./scripts", exist_ok=True)

In [25]:
# pip install --upgrade pip
# pip uninstall numpy
# pip install numpy
# pip install --upgrade numpy


In [1]:
# %%bash
# pip install -U "numpy==1.26.4" "pandas==2.2.2"
# python - <<'PY'
# import numpy as np, pandas as pd
# print("numpy", np.__version__)
# print("pandas", pd.__version__)
# PY
# pip install datasets
# pip install transformers datasets tokenizers accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 41.0 MB/s  0:00:00 eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4


numpy 1.26.4
pandas 2.2.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 536.6/536.6 kB 31.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 35.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 35.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.5/47.5 MB 41.5 MB/s  0:00:01 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17/17 [datasets]/17 [datasets]ce-hub]


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 25.3 MB/s  0:00:00 eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 27.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 21.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 791.8/791.8 kB 23.7 MB/s  0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface_hub 1.3.3
    Uninstalling huggingface_hub-1.3.3:
      Successfully uninstalled huggingface_hub-1.3.3
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6/6 [accelerate]6 [accelerate]s]ub]


In [2]:
import warnings
warnings.filterwarnings(
    "ignore",
    message="`tokenizer` is deprecated and will be removed in version 5.0.0"
)

## Check data

In [ ]:
import os
import pandas as pd

In [76]:
# pkl_path = "./data/news/news_network_1990_m0.0-M1.0.pkl"
# obj = pd.read_pickle(pkl_path)

# print(type(obj), obj.keys())
# G = obj["G"]
# print("new :nodes:", len(G.nodes()), "edges:", len(G.edges()))

# pkl_path = "./data/paper/paper_network_1990_m0.0-M0.5.pkl"
# obj = pd.read_pickle(pkl_path)

# print(type(obj), obj.keys())
# G = obj["G"]
# print("paper : nodes:", len(G.nodes()), "edges:", len(G.edges()))

In [75]:
DATA_ROOT = "./data"
YEARS = range(1990, 1996)

PKL_TPL = {
    "news":  "news_network_{year}_m0.0-M1.0.pkl",
    "paper": "paper_network_{year}_m0.0-M0.5.pkl",
}

rows = []

for domain, tpl in PKL_TPL.items():
    for year in YEARS:
        pkl_path = os.path.join(
            DATA_ROOT,
            domain,
            tpl.format(year=year)
        )

        if not os.path.exists(pkl_path):
            print(f"[SKIP] missing {pkl_path}")
            continue

        obj = pd.read_pickle(pkl_path)
        G = obj["G"]

        rows.append({
            "domain": domain,
            "year": year,
            "num_nodes": G.number_of_nodes(),
            "num_edges": G.number_of_edges(),
        })

df = pd.DataFrame(rows).sort_values(["domain", "year"])
df


,domain,year,num_nodes,num_edges
0,news,1990,2434,43851
1,news,1991,1664,15338
2,news,1992,2392,39561
3,news,1993,2437,38124
4,news,1994,2642,46871
5,news,1995,2658,41287
6,paper,1990,12593,397239
7,paper,1991,12657,417466
8,paper,1992,12986,445814
9,paper,1993,13079,461959


## Code Run

### Build vocab & tokenizer

In [7]:
%%bash
python ./scripts/build_vocab.py \
  --data_root ./data \
  --years 1990 1991 1992 1993 1994 1995 \
  --out_vocab ./tokenizer/concept_vocab_encoded.txt \
  --out_mapping ./tokenizer/node_mapping.json \
  --out_common_dir ./outputs/common_node_by_year

In [8]:
%%bash
python scripts/build_tokenizer.py \
  --vocab_txt ./tokenizer/concept_vocab_encoded.txt \
  --out_dir ./tokenizer

[OK] tokenizer.json saved_to=./tokenizer/tokenizer.json
[OK] vocab_size=22582 (incl specials=5)


### Make walk corpus & fit to HuggingFace dataset

In [64]:
%%bash
for Y in 1990 1991 1992 1993 1994 1995; do
  python scripts/make_walk_corpus_encoded.py \
    --pkl ./data/news/news_network_${Y}_m0.0-M1.0.pkl \
    --out ./corpus/news_${Y}.txt \
    --walk_length 20 --num_walks 20000 --seed 42 \
    --start_mode uniform \
    --mapping_json ./tokenizer/news_mapping_${Y}.json

  python scripts/make_walk_corpus_encoded.py \
    --pkl ./data/paper/paper_network_${Y}_m0.0-M0.5.pkl \
    --out ./corpus/paper_${Y}.txt \
    --walk_length 20 --num_walks 20000 --seed 42 \
    --start_mode uniform \
    --mapping_json ./tokenizer/paper_mapping_${Y}.json
done

[OK] start_mode=uniform wrote 20000 walks to ./corpus/news_1990.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/paper_1990.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/news_1991.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/paper_1991.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/news_1992.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/paper_1992.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/news_1993.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/paper_1993.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/news_1994.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/paper_1994.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/news_1995.txt
[OK] start_mode=uniform wrote 20000 walks to ./corpus/paper_1995.txt


In [65]:
%%bash
python scripts/make_hf_dataset.py \
  --corpus_root ./corpus \
  --out_dir ./datasets \
  --years 1990 1991 1992 1993 1994 1995 \
  --domains news paper

Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 1192834.41 examples/s]


[OK] saved news_1990 -> ./datasets/news_1990 (n=20000)


Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 1119750.12 examples/s]


[OK] saved paper_1990 -> ./datasets/paper_1990 (n=20000)


Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 1218194.33 examples/s]


...

[OK] saved paper_1994 -> ./datasets/paper_1994 (n=20000)


Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 1114957.80 examples/s]


[OK] saved news_1995 -> ./datasets/news_1995 (n=20000)


Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 1200722.56 examples/s]


[OK] saved paper_1995 -> ./datasets/paper_1995 (n=20000)


### Train (baseline)

In [83]:
%%bash
y=1990

python scripts/train_mlm.py \
  --dataset_dir ./datasets/news_${y} \
  --tokenizer_json ./tokenizer/tokenizer.json \
  --out_dir ./checkpoints/seed/news_${y}_seed \
  --epochs 6 \
  --lr 4e-4 \
  --mlm_prob 0.15

Map: 100%|██████████| 1000/1000 [00:00<00:00, 7526.09 examples/s]
/root/scripts/train_mlm.py:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Start training


100%|██████████| 3564/3564 [01:33<00:00, 38.14it/s]


{'loss': 9.1738, 'grad_norm': 2.2826614379882812, 'learning_rate': 0.00022122905027932964, 'epoch': 0.17}
{'loss': 7.3108, 'grad_norm': 1.7600152492523193, 'learning_rate': 0.0003976366322008863, 'epoch': 0.34}
{'loss': 7.2004, 'grad_norm': 1.8385194540023804, 'learning_rate': 0.0003858197932053176, 'epoch': 0.51}
{'loss': 7.0986, 'grad_norm': 2.093022346496582, 'learning_rate': 0.00037400295420974894, 'epoch': 0.67}
{'loss': 6.9368, 'grad_norm': 2.148613214492798, 'learning_rate': 0.00036218611521418023, 'epoch': 0.84}
{'loss': 6.8425, 'grad_norm': 2.4015398025512695, 'learning_rate': 0.0003503692762186115, 'epoch': 1.01}
{'loss': 6.7783, 'grad_norm': 2.0191879272460938, 'learning_rate': 0.00033855243722304287, 'epoch': 1.18}
{'loss': 6.6596, 'grad_norm': 1.9301304817199707, 'learning_rate': 0.00032673559822747416, 'epoch': 1.35}
{'loss': 6.6467, 'grad_norm': 2.1138482093811035, 'learning_rate': 0.0003149187592319055, 'epoch': 1.52}
{'loss': 6.5914, 'grad_norm': 2.0722756385803223, 'l

100%|██████████| 32/32 [00:00<00:00, 88.84it/s]


[OK] eval_loss=6.04583215713501  perplexity=422.35
[OK] trained & saved to ./checkpoints/seed/news_1990_seed


In [86]:
%%bash
y=1990

python scripts/train_mlm.py \
  --dataset_dir ./datasets/paper_${y} \
  --tokenizer_json ./tokenizer/tokenizer.json \
  --out_dir ./checkpoints/seed/news_${y}_seed \
  --epochs 6 \
  --lr 2.5e-4 \
  --mlm_prob 0.15

Map: 100%|██████████| 1000/1000 [00:00<00:00, 6939.01 examples/s]
/root/scripts/train_mlm.py:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Start training


100%|██████████| 3564/3564 [01:37<00:00, 36.47it/s]


{'loss': 9.6436, 'grad_norm': 2.086193084716797, 'learning_rate': 0.000138268156424581, 'epoch': 0.17}
{'loss': 8.5045, 'grad_norm': 1.9315451383590698, 'learning_rate': 0.0002485228951255539, 'epoch': 0.34}
{'loss': 8.3069, 'grad_norm': 2.0612058639526367, 'learning_rate': 0.0002411373707533235, 'epoch': 0.51}
{'loss': 8.1866, 'grad_norm': 1.913453221321106, 'learning_rate': 0.00023375184638109307, 'epoch': 0.67}
{'loss': 8.0901, 'grad_norm': 2.548715353012085, 'learning_rate': 0.00022636632200886264, 'epoch': 0.84}
{'loss': 8.0648, 'grad_norm': 2.159560203552246, 'learning_rate': 0.0002189807976366322, 'epoch': 1.01}
{'loss': 7.9273, 'grad_norm': 2.5530595779418945, 'learning_rate': 0.00021159527326440177, 'epoch': 1.18}
{'loss': 7.885, 'grad_norm': 3.1937429904937744, 'learning_rate': 0.00020420974889217135, 'epoch': 1.35}
{'loss': 7.8012, 'grad_norm': 3.9805943965911865, 'learning_rate': 0.00019682422451994093, 'epoch': 1.52}
{'loss': 7.7367, 'grad_norm': 3.0107405185699463, 'learn

100%|██████████| 32/32 [00:00<00:00, 80.74it/s]


[OK] eval_loss=7.168048858642578  perplexity=1297.31
[OK] trained & saved to ./checkpoints/seed/news_1990_seed


### Train (annual)

In [87]:
%%bash
python scripts/train_annual_2stage.py \
  --domain news \
  --dataset_root ./datasets \
  --tokenizer_json ./tokenizer/tokenizer.json \
  --ckpt_root ./checkpoints/annual_node \
  --years 1990 1991 1992 1993 1994 1995 \
  --seed_ckpt ./checkpoints/seed/news_1990_seed \
  --eval_ratio 0.05 \
  --stage1_mlm_prob 0.15 \
  --stage1_epochs 1 \
  --stage1_lr 5e-4 \
  --stage2_epochs 1 \
  --stage2_lr 2e-4 \
  --stage2_exact_k_masks 1

Traceback (most recent call last):
  File "/root/scripts/train_annual_2stage_preinit.py", line 320, in <module>
    main()
  File "/root/scripts/train_annual_2stage_preinit.py", line 220, in main
    raise FileNotFoundError(f"seed_ckpt not found: {seed_path}")
FileNotFoundError: seed_ckpt not found: ./checkpoints/seed_preinit/news_1990_bertbase


CalledProcessError: Command 'b'init_y=1990\n\npython scripts/train_annual_2stage_preinit.py \\\n  --domains news \\\n  --years 1991 1992 1993 1994 1995 \\\n  --dataset_root ./datasets \\\n  --ckpt_root ./checkpoints/annual_node_preinit \\\n  --seed_ckpt "./checkpoints/seed_preinit/{domain}_${init_y}_bertbase" \\\n  --stage both \\\n  --stage1_epochs 1 --stage1_lr 5e-4 --stage1_mlm_prob 0.15 \\\n  --stage2_epochs 1 --stage2_lr 2e-4 --stage2_exact_k_masks 1 \\\n  --eval_ratio 0.05 \\\n  --eval_stage1\n'' returned non-zero exit status 1.

### 거리에 따라 

In [70]:
%%bash
python scripts/find_common_targets_from_corpus.py \
  --corpus_root ./corpus \
  --years 1990 1991 1992 1993 1994 1995 \
  --min_count 5 \
  --out_dir ./outputs/common_targets

[OK] year=1990 common=261 saved: ./outputs/common_targets/common_targets_1990.txt
[OK] year=1991 common=167 saved: ./outputs/common_targets/common_targets_1991.txt
[OK] year=1992 common=298 saved: ./outputs/common_targets/common_targets_1992.txt
[OK] year=1993 common=285 saved: ./outputs/common_targets/common_targets_1993.txt
[OK] year=1994 common=298 saved: ./outputs/common_targets/common_targets_1994.txt
[OK] year=1995 common=319 saved: ./outputs/common_targets/common_targets_1995.txt


In [71]:
import pandas as pd

df = pd.read_csv(
    "./outputs/common_targets/common_targets_1990.tsv",
    sep="\t"
)

display(df.sort_values(["total_count"], ascending=False).head(10))
display(df.sort_values(["news_count", "paper_count"], ascending=False).head(10))
display(df.sort_values(["paper_count", "news_count"], ascending=False).head(10))

,rank,token,news_count,paper_count,total_count
0,1,automation,3650,172,3822
1,2,software,1948,449,2397
2,3,office_automation,931,7,938
3,4,politics,43,788,831
4,5,manufacturing,797,17,814
5,6,productivity,568,97,665
6,7,cost_control,633,10,643
7,8,systems_design,510,52,562
8,9,financial_services,468,12,480
9,10,data_processing,411,67,478


,rank,token,news_count,paper_count,total_count
0,1,automation,3650,172,3822
1,2,software,1948,449,2397
2,3,office_automation,931,7,938
4,5,manufacturing,797,17,814
6,7,cost_control,633,10,643
5,6,productivity,568,97,665
7,8,systems_design,510,52,562
8,9,financial_services,468,12,480
10,11,revenue,415,36,451
9,10,data_processing,411,67,478


,rank,token,news_count,paper_count,total_count
3,4,politics,43,788,831
1,2,software,1948,449,2397
11,12,population,12,437,449
21,22,visualization,19,281,300
23,24,fuzzy_logic,22,252,274
14,15,architecture,162,210,372
0,1,automation,3650,172,3822
25,26,cmos,117,143,260
29,30,health_care,111,133,244
44,45,magnetic_resonance_imaging,20,131,151


In [77]:
%%bash
python scripts/predict_neighbors_by_distance.py \
  --years 1991 1992 1993 1994 1995 \
  --domains news paper \
  --ckpt_root ./checkpoints/annual_node \
  --stage stage2 \
  --corpus_root ./corpus \
  --tokenizer_json ./tokenizer/tokenizer.json \
  --targets automation software	productivity data_processing architecture politics health_care cmos \
  --distance 10 \
  --topk 30 \
  --max_contexts 1000 \
  --out_root ./outputs/mask_pred

[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=automation.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=software.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=productivity.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=data_processing.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=architecture.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=politics.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=health_care.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1991/target=cmos.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1992/target=automation.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1992/target=software.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1992/target=productivity.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1992/target=data_processing.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1992/target=architecture.tsv
[OK] saved ./outputs/mask_pred/dist=10/year=1992/target=politics.tsv
[

In [78]:
targets = [
    "automation", "software", "productivity", "data_processing", "architecture", "politics", "health_care",
]
base = "./outputs/mask_pred/dist=10/year=1991/"

for t in targets:
    path = f"{base}/target={t}.tsv"
    print(f"\n============================== TARGET: {t} ==============================")
    df = pd.read_csv(path, sep="\t")
    display(df[cols].head(20))



============================== TARGET: automation ==============================


,rank,news_neighbor,news_avg_prob,paper_neighbor,paper_avg_prob
0,1,electronic_information_systems,0.029115,mathematics,0.009935
1,2,automation,0.028820,philosophy,0.006845
2,3,telephones,0.016107,physics,0.006261
3,4,bank_automation,0.012668,biology,0.005255
4,5,computers,0.010992,linguistics,0.004469
5,6,banks,0.010289,geometry,0.004395
6,7,software,0.010282,management,0.003916
7,8,automated_teller_machines,0.010147,genetic_algorithm,0.003135
8,9,banking_industry,0.009747,graph_theory,0.003019
9,10,electric_light_and_power,0.009743,psychology,0.003009



============================== TARGET: software ==============================


,rank,news_neighbor,news_avg_prob,paper_neighbor,paper_avg_prob
0,1,electronic_information_systems,0.032756,mathematics,0.009626
1,2,automation,0.025114,philosophy,0.006541
2,3,telephones,0.016860,physics,0.005942
3,4,apparel,0.013468,biology,0.005007
4,5,bank_automation,0.012946,linguistics,0.004288
5,6,banks,0.010514,geometry,0.004195
6,7,computers,0.010344,management,0.003772
7,8,software,0.010212,genetic_algorithm,0.003031
8,9,automated_teller_machines,0.009476,graph_theory,0.002996
9,10,stocks_and_bonds,0.009399,psychology,0.002934



============================== TARGET: productivity ==============================


...

,rank,news_neighbor,news_avg_prob,paper_neighbor,paper_avg_prob
0,1,automation,0.025897,mathematics,0.010241
1,2,telephones,0.022090,philosophy,0.007093
2,3,labor,0.016113,physics,0.006302
3,4,electronic_information_systems,0.012274,biology,0.005137
4,5,stocks_and_bonds,0.012220,linguistics,0.004459
5,6,bank_automation,0.012084,geometry,0.004300
6,7,automated_teller_machines,0.011696,management,0.004003
7,8,navies,0.011106,psychology,0.003116
8,9,robots,0.011070,genetic_algorithm,0.003067
9,10,news_and_news_media,0.010521,graph_theory,0.003020



============================== TARGET: politics ==============================


,rank,news_neighbor,news_avg_prob,paper_neighbor,paper_avg_prob
0,1,automation,0.026090,mathematics,0.009458
1,2,apparel,0.018417,philosophy,0.006430
2,3,bank_automation,0.012774,physics,0.005835
3,4,automated_teller_machines,0.012468,biology,0.005049
4,5,computers,0.010306,linguistics,0.004241
5,6,banks,0.009921,geometry,0.004204
6,7,software,0.009584,management,0.003621
7,8,stocks_and_bonds,0.009338,genetic_algorithm,0.003017
8,9,banking_industry,0.008843,graph_theory,0.002984
9,10,electronic_information_systems,0.008771,psychology,0.002886



============================== TARGET: health_care ==============================


,rank,news_neighbor,news_avg_prob,paper_neighbor,paper_avg_prob
0,1,electronic_information_systems,0.059321,mathematics,0.009347
1,2,telephones,0.033977,philosophy,0.006618
2,3,automation,0.024290,physics,0.006005
3,4,bank_automation,0.012964,biology,0.005009
4,5,robots,0.011919,linguistics,0.004245
5,6,computers,0.011551,geometry,0.004141
6,7,suits_and_litigation,0.011231,management,0.003693
7,8,marketing_and_merchandising,0.010486,genetic_algorithm,0.003004
8,9,news_and_news_media,0.010409,graph_theory,0.002934
9,10,stocks_and_bonds,0.010329,psychology,0.002933
